In [ ]:
%%bash

#################################################################################################
# working directory
W_DIR="/home/gpfs/o_kumpitch/05_LosingArchaea/00_data/01_Pasolli_Madagascar_Ethiopia/" 

# input directory
#I_DIR=$W_DIR"Data/" 
I_DIR="/home/gpfs/o_kumpitch/05_LosingArchaea/00_data/01_Pasolli_Madagascar_Ethiopia/03_batch3/pe/"
# name of the output directory
#NAME="Results" 
NAME="000_KRAKEN2/03_batch3/02_UGHH_pe"

#create output directories
mkdir $W_DIR$NAME
mkdir $W_DIR$NAME'/main_output'
mkdir $W_DIR$NAME'/bash_scripts'
mkdir $W_DIR$NAME'/std_out'

# sbatch scripts directory, main output and standard output/error
B_DIR=$W_DIR$NAME"/bash_scripts/" 
OUT_D=$W_DIR$NAME'/main_output/'
STD_OUT=$W_DIR$NAME'/std_out/'

# setup number of cores
thr=12

# setup memory
mem=50G

# setup time
time=24:00:00

# exclude nodes
nodes=sx138,sx037,sx038

# source conda env
conda_env=". /home/conda/miniconda3/bin/activate kraken_2.1.2"

#################################################################################################

#iterate over input directory
for i in $(ls -h $I_DIR); do
R1=${i//_2.fastq.gz/_1.fastq.gz}
R2=${i//_1.fastq.gz/_2.fastq.gz}
string=${i%%/} export string
	# check if .fastq.gz file
    if [[ $string == *"fastq.gz"* ]]; then
        arrIN=(${string//_*.fastq.gz/ })
        echo ${arrIN[0]}
        rm $B_DIR${arrIN[0]}".bash"
        echo "#!/bin/bash" >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH -p debug" >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH -N 1" >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH -n" $thr >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH -t" $time >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH --mem" $mem >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH --exclude="$nodes >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH -o $STD_OUT${arrIN[0]}.out" >> $B_DIR${arrIN[0]}".bash"
        echo "#SBATCH -e $STD_OUT${arrIN[0]}.err" >> $B_DIR${arrIN[0]}".bash"
        echo "" >> $B_DIR${arrIN[0]}".bash"
        echo $conda_env >> $B_DIR${arrIN[0]}".bash"
        echo "" >> $B_DIR${arrIN[0]}".bash"
        
        # edit command accordingly!!!!! Always add appropriate number of threads using parameter $thr
        #echo "metaphlan $I_DIR${arrIN[0]}.fastq --input_type fastq --output_file $OUT_D${arrIN[0]}_taxonomic_profile.tsv --samout $OUT_D${arrIN[0]}.sam --nproc $thr" >> $B_DIR${arrIN[0]}".bash"
        echo "kraken2 --db /home/conda/int_microbiome/databases/kraken2/kraken2_db_uhgg_v2.0.1/ $I_DIR$R1 $I_DIR$R2 --paired --confidence 0.1 --gzip-compressed --memory-mapping --threads $thr --output $OUT_D${arrIN[0]}_kraken2.out --use-names --report $OUT_D${arrIN[0]}_report.out" >> $B_DIR${arrIN[0]}".bash"
        
        # check sample.bash script before uncomenting this and submitting job
        sbatch $B_DIR${arrIN[0]}".bash"
    fi
done